# Book distribution


```{custom_download_link} ./linear_constrained_optimization_stripped.ipynb
:replace_default: "True"
```

Click {fa}`rocket` --> {guilabel}`Live Code` to activate live coding on this page!

## Problem
We'll consider planning the shipment of books from distribution centers to stores where they are needed. There are three distribution centers at different cities: Amsterdam, Den Haag and Rotterdam. They have $250$, $130$ and $235$ $\text{books}$. There are four stores in Haarlem, Utrecht, Delft and Breda. They ordered $70$, $230$, $240$ and $70$ $\text{books}$. All stores can receive books from all distribution centers, as shown below:

<iframe width="100%" height="600" frameborder="0" allowfullscreen allow="geolocation" src="//umap.openstreetmap.fr/nl/map/linear-constrained-optimization-problem_1054820?scaleControl=false&miniMap=false&scrollWheelZoom=false&zoomControl=false&editMode=disabled&moreControl=false&searchControl=false&tilelayersControl=false&embedControl=false&datalayersControl=false&onLoadPanel=none&captionBar=false&captionMenus=false&fullscreenControl=false&locateControl=false&measureControl=false&editinosmControl=false&starControl=false&datalayers=b16d6f26-9a8c-4e85-8448-9f44b6b4131e#9/52.0234/4.7324"></iframe>

There are the following costs in $\text{€ of transportation per book}$: 

|     From   \ To       |     Haarlem    |     Utrecht    |     Delft    |      Breda     |
|:---------------------:|:--------------:|:--------------:|:------------:|:--------------:|
|        Amsterdam      |      $ 7 $     |     $  11 $    |    $ 16 $    |     $ 26 $     |
|        Den Haag       |      $ 7 $     |     $  13 $    |    $  5 $    |     $ 20 $     |
|        Rotterdam      |      $16 $     |     $  28 $    |    $  7 $    |     $ 12 $     |

The goal is to minimize the shipping costs while meeting demand.

## Model
We need to define our model in the form of a linear constrained optimization model {eq}`linear_constrained_optimization` and to apply `scipy.optimize.linprog` in matrix notation {eq}`linear_constrained_optimization_vector`.

We'll define the model as follows:
- Design variables: how many books go from which distribution center to which store
- Objective function: minimum shippings costs 
- Inequality constraint functions: don't run out of stock in warehouses
- Equality constraint functions: meet demand of stores
- Bounds: book are only transported from distribution centers to stores.

### Design variables
Let's start with our design variables. In this case that could be the amount of books $x$ transported from each distribution center $i = \left[ 1 \left(\text{Amsterdam}\right),2 \left(\text{Den Haag}\right),3 \left( \text{Rotterdam}\right) \right] $ to each store $j = \left[ 1 \left( \text{Haarlem}\right),2  \left(\text{Utrecht}\right),3 \left(\text{Delft}\right),4 \left( \text{Breda} \right)\right] $:

```{math}
:label: linear_constrained_optimization_x_matrix
\begin{align}
& x_{ij}  = \left[ \begin{matrix}
{{x}_{\text{Amsterdam} \to \text{Haarlem}}} & {{x}_{\text{Amsterdam} \to \text{Utrecht}}} & {{x}_{\text{Amsterdam} \to \text{Delft}}} & {{x}_{\text{Amsterdam} \to \text{Breda}}}  \\
   {{x}_{\text{Den Haag} \to \text{Haarlem}}} & {{x}_{\text{Den Haag} \to \text{Utrecht}}} & {{x}_{\text{Den Haag} \to \text{Delft}}} & {{x}_{\text{Den Haag} \to \text{Breda}}}  \\
   {{x}_{\text{Rotterdam} \to \text{Haarlem}}} & {{x}_{\text{Rotterdam} \to \text{Utrecht}}} & {{x}_{\text{Rotterdam} \to \text{Delft}}} & {{x}_{\text{Rotterdam} \to \text{Breda}}} \end{matrix} \right] \\\\
& x_{ij} = \left[ \begin{matrix}
   {{x}_{11}} & {{x}_{12}} & {{x}_{13}} & {{x}_{14}}  \\
   {{x}_{21}} & {{x}_{22}} & {{x}_{23}} & {{x}_{24}}  \\
   {{x}_{31}} & {{x}_{32}} & {{x}_{33}} & {{x}_{34}}  \\
\end{matrix} \right]
\end{align}
```

:::{card} Test Yourself
<iframe src="https://tudelft.h5p.com/content/1292246306931991957/embed" aria-label="Design variable size" width="1088" height="637" frameborder="0" allowfullscreen="allowfullscreen" allow="autoplay *; geolocation *; microphone *; camera *; midi *; encrypted-media *"></iframe><script src="https://tudelft.h5p.com/js/h5p-resizer.js" charset="UTF-8"></script>
:::

Alternatively, the design variables can be expressed as follows:

```{math}
:label: linear_constrained_optimization_x_vector
\begin{align}
&x  = \left[ \begin{matrix}
   {{x}_{\text{Amsterdam} \to \text{Haarlem}}} \\ {{x}_{\text{Amsterdam} \to \text{Utrecht}}} \\ {{x}_{\text{Amsterdam} \to \text{Delft}}} \\ {{x}_{\text{Amsterdam} \to \text{Breda}}}  \\
   {{x}_{\text{Den Haag} \to \text{Haarlem}}} \\ {{x}_{\text{Den Haag} \to \text{Utrecht}}} \\ {{x}_{\text{Den Haag} \to \text{Delft}}} \\ {{x}_{\text{Den Haag} \to \text{Breda}}}  \\
   {{x}_{\text{Rotterdam} \to \text{Haarlem}}} \\ {{x}_{\text{Rotterdam} \to \text{Utrecht}}} \\ {{x}_{\text{Rotterdam} \to \text{Delft}}} \\ {{x}_{\text{Rotterdam} \to \text{Breda}}} \end{matrix} \right] \\\\
& x = \left[ \begin{matrix}
   {{x}_{11}} & {{x}_{12}} & {{x}_{13}} & {{x}_{14}}  &
   {{x}_{21}} & {{x}_{22}} & {{x}_{23}} & {{x}_{24}}  &
   {{x}_{31}} & {{x}_{32}} & {{x}_{33}} & {{x}_{34}}
\end{matrix} \right]^T
\end{align}
```

### Objective function
Now we can define the costs as the sum of the amount of books transported with the cost per transport as in {eq}`linear_constrained_optimization`in the form of $\mathop {\min }\limits_x f\left(x_{ij}\right) $:

```{math}
:label: linear_constrained_optimization_f
{\min }\limits_x f\left(x_{ij}\right) = 7x_{11} + 11x_{12} + 16x_{13} + 26x_{14} + 7x_{21} + 13x_{22} + 5x_{23} + 20x_{24} + 16x_{31} + 28x_{32} + 7x_{33} + 12x_{34}
```

This can be converted to matrix notation. In case of the design variables defined as in {eq}`linear_constrained_optimization_vector` in the form $\mathop {\min }\limits_x {c^T}x$ with $c$:

```{math}
:label: linear_constrained_optimization_c
c = {{\left[ \begin{matrix}
   7 & 11 & 16 & 26 & 7 & 13 & 5 & 20 & 16 & 28 & 7 & 12  \\
\end{matrix} \right]}^{T}}
```

:::{card} Test Yourself
<iframe src="https://tudelft.h5p.com/content/1292246313197632677/embed" aria-label="Objective function" width="1088" height="637" frameborder="0" allowfullscreen="allowfullscreen" allow="autoplay *; geolocation *; microphone *; camera *; midi *; encrypted-media *"></iframe><script src="https://tudelft.h5p.com/js/h5p-resizer.js" charset="UTF-8"></script>
:::

### Inequality constraints

Let's continue with the inequality constraints, which should deal with that the stock of the distribution centers should always be bigger than $0$. Or, stated differently, the sum of the amount of books transported out of each distribution center should be small or equal than the stock of each distribution center. These can be defined in the form ${{g}}\left(x_{ij}\right) \le 0$ as:

```{math}
:label: linear_constrained_optimization_g
g_1\left(x_{ij}\right) = x_{11} + x_{12} + x_{13} + x_{14} - 250 \\
g_2\left(x_{ij}\right) = x_{21} + x_{22} + x_{23} + x_{24} - 130 \\
g_3\left(x_{ij}\right) = x_{31} + x_{32} + x_{33} + x_{34} - 235
```

Or in matrix notation in the form ${A_{ub}}x \le {b_{ub}}$ as:
```{math}
:label: linear_constrained_optimization_g_matrix
\left[ \begin{matrix}
   1 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0  \\
   0 & 0 & 0 & 0 & 1 & 1 & 1 & 1 & 0 & 0 & 0 & 0  \\
   0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 1  \\
\end{matrix} \right] x \le \left[ \begin{matrix}
   250  \\
   130  \\
   235  \\
\end{matrix} \right]
```

:::{card} Test yourself
<iframe src="https://tudelft.h5p.com/content/1292246318200839587/embed" aria-label="Clone of Objective function" width="1088" height="637" frameborder="0" allowfullscreen="allowfullscreen" allow="autoplay *; geolocation *; microphone *; camera *; midi *; encrypted-media *"></iframe><script src="https://tudelft.h5p.com/js/h5p-resizer.js" charset="UTF-8"></script>
:::

### Equality constraints

Finally, let's define the inequality constraints, which should deal with that the each store receives the correct amount of books. Or, stated differently, the sum amount of paper books transported to each store should be equal to the demand of that store. These can be defined in the from ${h}\left(x_{ij}\right) = 0$ as:

```{math}
:label: linear_constrained_optimization_g_eq
h_1\left(x_{ij}\right) = x_{11} + x_{21} + x_{31} - 70 \\
h_2\left(x_{ij}\right) = x_{12} + x_{22} + x_{32} - 230 \\
h_3\left(x_{ij}\right) = x_{13} + x_{23} + x_{33} - 240 \\
h_3\left(x_{ij}\right) = x_{14} + x_{24} + x_{34} - 70
```

Or in matrix notation in the form ${A_{eq}}x = {b_{eq}}$ as:
```{math}
:label: linear_constrained_optimization_g_eq_matrix
\left[ \begin{matrix}
   1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 0  \\
   0 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 & 0  \\
   0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 0  \\
   0 & 0 & 0 & 1 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 1  \\
\end{matrix} \right]x=\left[ \begin{matrix}
   70  \\
   230  \\
   240  \\
   70  \\
\end{matrix} \right]
```

### Bounds

The number of books cannot be negative (assuming that book are only transported from distribution centers to stores). The maximum number of books transported could be the number of book available in a distribution center, but this is already defined by the constraint functions. Therefore, the bounds can be defined as:

```{math}
:label: bounds
0<{{x}_{i}}\text{   } i=1,2,...,12
```

## Method

Now let's solve this problem using an optimization method.

### Import libraries

As this problem is higher-dimensional, we cannot easily plot the solution space. Therefore, we won't import `matplotlib`.

In [1]:
import scipy as sp
import numpy as np

In [2]:
import scipy as sp 
import numpy as np

### Define variables

We can define all our variables as numpy array's. As before, the (length of) design variable itself doesn't have to be specified. For vectors, you don't have to care about them being row or column vectors.

To reduce the search space, the bounds are set between $0$ and the amount of books per distribution center.

In [2]:
# x is design variable array with size (12,)

bounds = np.array([[0, None],
          [0, None],
          [0, None],
          [0, None],
          [0, None],
          [0, None],
          [0, None],
          [0, None],
          [0, None],
          [0, None],
          [0, None],
          [0, None]])


A_ub = np.array([[1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0],
                [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0],
                [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]])
b_ub = np.array([250, 130, 235])

A_eq = np.array([[1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
                [0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0],
                [0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0],
                [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1]])
b_eq = np.array([70, 230, 240, 70])

### Define objective function

In [3]:
c = np.array([7, 11, 16, 26, 7, 13, 5, 20, 16, 28, 7, 12])

### Solve the problem

In [4]:
result = sp.optimize.linprog(c, A_ub, b_ub, A_eq, b_eq, bounds)
print(result)

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: 5380.0
              x: [ 2.000e+01  2.300e+02  0.000e+00  0.000e+00  5.000e+01
                   0.000e+00  8.000e+01  0.000e+00  0.000e+00  0.000e+00
                   1.600e+02  7.000e+01]
            nit: 7
          lower:  residual: [ 2.000e+01  2.300e+02  0.000e+00  0.000e+00
                              5.000e+01  0.000e+00  8.000e+01  0.000e+00
                              0.000e+00  0.000e+00  1.600e+02  7.000e+01]
                 marginals: [ 0.000e+00  0.000e+00  1.100e+01  1.600e+01
                              0.000e+00  2.000e+00  0.000e+00  1.000e+01
                              7.000e+00  1.500e+01  0.000e+00  0.000e+00]
          upper:  residual: [       inf        inf        inf        inf
                                    inf        inf        inf        inf
                                    inf        inf        inf 

## Exercise

:::{card}
Click {fa}`rocket` --> {guilabel}`Live Code` to activate live coding on this page.
<iframe src="https://tudelft.h5p.com/content/1292246941187853367/embed" aria-label="Activate constraints" width="1088" height="637" frameborder="0" allowfullscreen="allowfullscreen" allow="autoplay *; geolocation *; microphone *; camera *; midi *; encrypted-media *"></iframe><script src="https://tudelft.h5p.com/js/h5p-resizer.js" charset="UTF-8"></script>
:::

:::{card}
<iframe src="https://tudelft.h5p.com/content/1292246943302196447/embed" aria-label="Changing dehmand" width="1088" height="637" frameborder="0" allowfullscreen="allowfullscreen" allow="autoplay *; geolocation *; microphone *; camera *; midi *; encrypted-media *"></iframe><script src="https://tudelft.h5p.com/js/h5p-resizer.js" charset="UTF-8"></script>
:::

## Questions, discussions and comments
<script src="https://utteranc.es/client.js"
        repo="TeachBooks/engineering-systems-optimization"
        issue-term="title"
        theme="github-light"
        crossorigin="anonymous"
        async>
</script>